In [ ]:
import xml.etree.ElementTree as ET
import json, requests, time
from copy import copy
import pandas as pd

def make_data(things,c=0):
    key_file = {}
    # set to 1 to wipe the file and write headers for the first pass
    for data_field in things:
        if 'station' in data_field:
            key_file = data_field['station']
        if 'dish' not in data_field:
            continue
        data_field['dish']['station'] = key_file['name']
        data_field['dish']['friendlyName'] = key_file['friendlyName']
        data_field['dish']['timeUTC'] = key_file['timeUTC']
        data_field['dish']['timeZoneOffset'] = key_file['timeZoneOffset']
        new_df = pd.DataFrame([data_field['dish']])
        new_df['_text'] = data_field['dish']['upSignal'][0]['spacecraft']
        ds_df = pd.DataFrame(data_field['dish']['downSignal'])
        us_df = pd.DataFrame(data_field['dish']['upSignal'])
        ts_df = pd.DataFrame(data_field['dish']['target'])
        new_df2 = pd.merge(new_df, ds_df, how='left', left_on='_text', right_on='spacecraft')
        new_df3 = pd.merge(new_df2, us_df, how='left', left_on='_text', right_on='spacecraft')
        new_df4 = pd.merge(new_df3, ts_df, how='left', left_on='_text', right_on='name')
        if(c == 1):
            with open('sample.csv', 'w+') as f:
                new_df4.to_csv(f,index=False, header=True)
            c = 0
        else:
            with open('sample.csv', 'a+') as f:
                new_df4.to_csv(f,index=False, header=False)

def dictify(r,root=True): # https://stackoverflow.com/questions/2148119/how-to-convert-an-xml-string-to-a-dictionary-in-python/30923963#30923963
    if root:
        return {r.tag : dictify(r, False)}
    d=copy(r.attrib)
    if r.text:
        d["_text"]=r.text
    for x in r.findall("./*"):
        if x.tag not in d:
            d[x.tag]=[]
        d[x.tag].append(dictify(x,False))
    return d

In [ ]:

N = 1365107113
#Fresh start of the program use c = 1, restarting the program use c = 0 if the file already exists with headers
c = 1
while(True):
    time.sleep(3)
    N += 5
    url   = "https://eyes.nasa.gov/dsn/data/" + str(N) + ".xml"
    try:
        print('INFO - Downloading Data',N)
        r = requests.get(url)
        if(r.status_code==200):
            print('INFO - Downloaded Data Successfully',N)
            text = r.text
            a = ET.fromstring(text)
            things = [dictify(x) for x in a]
            make_data(things,c)
            c = 0
        else:
            raise Exception(r.status_code)
    except Exception as e:
        print('ERROR',e,N)